In [1]:
import pandas as pd
import xarray as xr
from IPython.display import clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
from ar6_ch6_rcmipfigs.constants import INPUT_DATA_DIR, OUTPUT_DATA_DIR, RESULTS_DIR

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_in


In [2]:
# name of output variable
name_deltaT = 'Delta T'

climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'
percentile = 'percentile'

### Define variables to look at:

In [3]:
# variables to include:

variables_erf_comp = [
    'ch4',
    'aerosol-total-with_bc-snow',
    'aerosol-radiation_interactions',
    'aerosol-cloud_interactions',
    'aerosol-total',
    'o3',
    'HFCs',
    # 'F-Gases|HFC',
    'bc_on_snow',
    'total_anthropogenic',
    'total',
]
variables_in_sum = [
    'aerosol-total-with_bc-snow',
    'ch4',
    # 'aerosol-radiation_interactions',
    # 'aerosol-cloud_interactions',
    #'aerosol-total',
    'o3',
    'HFCs',
    #'bc_on_snow'
]
# total ERFs for anthropogenic and total:
variables_erf_tot = ['total_anthropogenic',
                     'total']
variables_all = variables_erf_comp + variables_erf_tot
# Scenarios to plot:
scenarios_fl = ['ssp534-over', 'ssp119', 'ssp460', 'ssp585', 'ssp370',
                'ssp370-lowNTCF-aerchemmip', 'ssp126', 'ssp245', 'ssp434',
                'ssp370-lowNTCF-gidden'
                ]

In [4]:
recommendation = 'recommendation'
IRFpercentiles = [recommendation]
# {'ECS = 2K':0.526, 'ECS = 3.4K':0.884, 'ECS = 5K': 1.136 }

Year to integrate from and to:

In [5]:
first_y = 1750
last_y = 2100

**Set reference year for temperature change:**

In [6]:
ref_year = 2020

**Years to output change in**

In [7]:
years = [2040, 2100]

### Input dataset:

In [8]:
PATH_DT_INPUT = OUTPUT_DATA_DIR / 'dT_data_RCMIP_recommendation.nc'

In [9]:
PATH_DT_TAB_OUTPUT = RESULTS_DIR / 'tables_recommendation' / 'table_sens_dT_cs_recommandetion.csv'







In [10]:
print(PATH_DT_INPUT)

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_out/dT_data_RCMIP_recommendation.nc


In [11]:
    
    
ds_DT = xr.open_dataset(PATH_DT_INPUT)

In [12]:
ds_DT.variable

<xarray.DataArray 'variable' (variable: 19)>
array(['co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
       'contrails', 'aerosol-radiation_interactions',
       'aerosol-cloud_interactions', 'bc_on_snow', 'land_use', 'volcanic',
       'solar', 'total_anthropogenic', 'total_natural', 'total',
       'aerosol-total', 'aerosol-total-with_bc-snow', 'HFCs'], dtype=object)
Coordinates:
  * variable  (variable) object 'co2' 'ch4' ... 'HFCs'

## Table

### Setup table:

In [13]:

iterables = [list(IRFpercentiles), years]


def setup_table(scenario_n='', variables=variables_all):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    table = pd.DataFrame(columns= variables, index=_i).transpose()
    table.index.name = scenario_n
    return table

In [14]:
# Dicitonary of tables with different ESC:
scntab_dic = {}
for scn in scenarios_fl:
    # Loop over scenrarios
    tab = setup_table(scenario_n=scn, variables=variables_erf_comp)  # make table
    for var in variables_erf_comp:
        # Loop over variables
        tabvar = var.split('|')[-1]
        dtvar = name_deltaT
        for key in IRFpercentiles:
            # Loop over ESC parameters
            for year in years:
                
                da_ref_y = ds_DT[dtvar].sel(
                    percentile=key,
                    scenario=scn, 
                    year=ref_year,
                ).squeeze()
                
                da_sel_y =ds_DT[dtvar].sel(
                    percentile=key,
                    scenario=scn, 
                    year=year
                ) 
                _tab_da = da_sel_y - da_ref_y
                a = float(_tab_da.loc[var].squeeze().values)
                tab.loc[tabvar, (key, year)] = a
    scntab_dic[scn] = tab.copy()

In [15]:
from IPython.display import display

for key in scntab_dic:
    display(scntab_dic[key])

recommendation            
                                         2040        2100
ssp534-over                                              
ch4                                 0.0656587   -0.167303
aerosol-total-with_bc-snow           0.123031    0.360511
aerosol-radiation_interactions      0.0346348 -0.00207481
aerosol-cloud_interactions          0.0867517    0.403812
aerosol-total                        0.121387    0.401737
o3                                  0.0425019   -0.132867
HFCs                                0.0422546   0.0151856
bc_on_snow                         0.00164463  -0.0412262
total_anthropogenic                  0.849015    0.882265
total                                0.804588    0.824326

recommendation            
                                         2040        2100
ssp119                                                   
ch4                                -0.0327327   -0.163053
aerosol-total-with_bc-snow           0.273961    0.424652
aerosol-radiation_interactions      0.0301799   0.0392254
aerosol-cloud_interactions           0.271757    0.426936
aerosol-total                        0.301937    0.466161
o3                                 -0.0830039   -0.148487
HFCs                               0.00542323 -0.00377154
bc_on_snow                         -0.0279765  -0.0415094
total_anthropogenic                  0.397892    0.115533
total                                0.353464   0.0575948

recommendation           
                                         2040       2100
ssp460                                                  
ch4                                 0.0865132   0.178782
aerosol-total-with_bc-snow        -0.00588118   0.178702
aerosol-radiation_interactions       0.015449 -0.0217373
aerosol-cloud_interactions         -0.0355464    0.22299
aerosol-total                      -0.0200974   0.201252
o3                                  0.0557647  0.0934309
HFCs                                0.0380702   0.040923
bc_on_snow                          0.0142162 -0.0225505
total_anthropogenic                  0.645107    2.25871
total                                 0.60068    2.20077

recommendation            
                                         2040        2100
ssp585                                                   
ch4                                 0.0616757    0.147175
aerosol-total-with_bc-snow           0.123031    0.232538
aerosol-radiation_interactions      0.0346348 -0.00605205
aerosol-cloud_interactions          0.0867517     0.26821
aerosol-total                        0.121387    0.262158
o3                                  0.0399694   0.0504914
HFCs                                0.0422471    0.223037
bc_on_snow                         0.00164463  -0.0296192
total_anthropogenic                  0.836005     4.10884
total                                0.791578      4.0509

recommendation            
                                         2040        2100
ssp370                                                   
ch4                                 0.0760285    0.276137
aerosol-total-with_bc-snow         -0.0516312  -0.0261019
aerosol-radiation_interactions      0.0130405  0.00024477
aerosol-cloud_interactions         -0.0852626  -0.0399591
aerosol-total                       -0.072222  -0.0397144
o3                                  0.0697092    0.235987
HFCs                                0.0264763   0.0968782
bc_on_snow                          0.0205908   0.0136124
total_anthropogenic                  0.663032     3.14749
total                                0.618605     3.08956

recommendation           
                                         2040       2100
ssp370-lowNTCF-aerchemmip                               
ch4                                 0.0760285   0.276137
aerosol-total-with_bc-snow          0.0872021   0.285005
aerosol-radiation_interactions      0.0196311  0.0170795
aerosol-cloud_interactions          0.0695543   0.295329
aerosol-total                       0.0891854   0.312409
o3                                  0.0244264   0.157365
HFCs                                0.0264763  0.0968782
bc_on_snow                        -0.00198331 -0.0274044
total_anthropogenic                  0.756627    3.36902
total                                  0.7122    3.31108

recommendation            
                                         2040        2100
ssp126                                                   
ch4                                -0.0207931   -0.155336
aerosol-total-with_bc-snow           0.228478    0.434201
aerosol-radiation_interactions      0.0256415   0.0478225
aerosol-cloud_interactions           0.226138    0.425765
aerosol-total                         0.25178    0.473588
o3                                 -0.0598128   -0.147167
HFCs                                0.0170798 -0.00145539
bc_on_snow                         -0.0233014  -0.0393862
total_anthropogenic                  0.505479    0.543195
total                                0.461052    0.485256

recommendation            
                                         2040        2100
ssp245                                                   
ch4                                 0.0417004 -0.00122657
aerosol-total-with_bc-snow          0.0469643    0.282241
aerosol-radiation_interactions     0.00418958  0.00472453
aerosol-cloud_interactions          0.0451808    0.313446
aerosol-total                       0.0493704     0.31817
o3                                  0.0149274  -0.0460715
HFCs                                0.0284388   0.0575121
bc_on_snow                        -0.00240609  -0.0359289
total_anthropogenic                  0.584383     1.75184
total                                0.539956     1.69391

recommendation            
                                         2040        2100
ssp434                                                   
ch4                                 0.0552716    0.124907
aerosol-total-with_bc-snow          0.0758486    0.260756
aerosol-radiation_interactions      0.0304127  0.00135336
aerosol-cloud_interactions           0.039147    0.286941
aerosol-total                       0.0695597    0.288294
o3                                  0.0256072   0.0466648
HFCs                               0.00340588 -0.00734877
bc_on_snow                         0.00628888  -0.0275375
total_anthropogenic                  0.530226     1.08706
total                                0.485799     1.02912

recommendation           
                                         2040       2100
ssp370-lowNTCF-gidden                                   
ch4                                  -0.03604  -0.122403
aerosol-total-with_bc-snow          0.0872021   0.285005
aerosol-radiation_interactions      0.0196311  0.0170795
aerosol-cloud_interactions          0.0695543   0.295329
aerosol-total                       0.0891854   0.312409
o3                                 -0.0440883 -0.0986165
HFCs                                0.0264745  0.0836752
bc_on_snow                        -0.00198331 -0.0274044
total_anthropogenic                  0.566567    2.64512
total                                 0.52214    2.58719

### Make table with all scenarios:

In [16]:
iterables = [list(IRFpercentiles), years]
iterables2 = [scenarios_fl, variables_erf_comp]


def setup_table2():  # scenario_n=''):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    _r = pd.MultiIndex.from_product(iterables2, names=['', ''])

    table = pd.DataFrame(columns=_r, index=_i).transpose()
    return table

In [17]:
tab = setup_table2()  # scenario_n=scn)

for scn in scenarios_fl:
    for var in variables_erf_comp:
        tabvar = var#.split('|')[-1]
        dtvar = name_deltaT
        for key in IRFpercentiles:
            for year in years:
                # compute difference between year and ref year
                _da_y = ds_DT[dtvar].sel(
                    percentile=key,
                    scenario=scn, 
                    year=year, 
                    variable=var
                ).squeeze()  # .squeeze()
                _da_refy = ds_DT[dtvar].sel(
                    scenario=scn, 
                    year=ref_year, 
                    variable=var,
                ).squeeze()
                # _tab_da = dic_ds[key][dtvar].sel(scenario=scn, time=slice(year,year))-  dic_ds[key][dtvar].sel(scenario=scn, time=slice(ref_year,ref_year)).squeeze()
                _tab_da = _da_y - _da_refy

                tab.loc[(scn, tabvar), (key, year)] = _tab_da.squeeze().values  # [0]

In [18]:
tab

recommendation  \
                                                                        2040   
                                                                               
ssp534-over           ch4                                0.06565872410380408   
                      aerosol-total-with_bc-snow         0.12303113815638728   
                      aerosol-radiation_interactions     0.03463483149905226   
                      aerosol-cloud_interactions         0.08675167458174482   
                      aerosol-total                      0.12138650608079715   
...                                                                      ...   
ssp370-lowNTCF-gidden o3                                -0.04408828493767006   
                      HFCs                              0.026474525052103126   
                      bc_on_snow                      -0.0019833087420466286   
                      total_anthropogenic                 0.5665673227761332   
                      total                               0.5221400921978727   

                                                                              
                                                                        2100  
                                                                              
ssp534-over           ch4                               -0.16730309506416727  
                      aerosol-total-with_bc-snow          0.3605109949745037  
                      aerosol-radiation_interactions  -0.0020748083021771557  
                      aerosol-cloud_interactions         0.40381198215439673  
                      aerosol-total                       0.4017371738522196  
...                                                                      ...  
ssp370-lowNTCF-gidden o3                                -0.09861654200626929  
                      HFCs                               0.08367522537429371  
                      bc_on_snow                       -0.027404367550451728  
                      total_anthropogenic                 2.6451235538746403  
                      total                                2.587185062896853  

[100 rows x 2 columns]

In [19]:
tab

recommendation  \
                                                                        2040   
                                                                               
ssp534-over           ch4                                0.06565872410380408   
                      aerosol-total-with_bc-snow         0.12303113815638728   
                      aerosol-radiation_interactions     0.03463483149905226   
                      aerosol-cloud_interactions         0.08675167458174482   
                      aerosol-total                      0.12138650608079715   
...                                                                      ...   
ssp370-lowNTCF-gidden o3                                -0.04408828493767006   
                      HFCs                              0.026474525052103126   
                      bc_on_snow                      -0.0019833087420466286   
                      total_anthropogenic                 0.5665673227761332   
                      total                               0.5221400921978727   

                                                                              
                                                                        2100  
                                                                              
ssp534-over           ch4                               -0.16730309506416727  
                      aerosol-total-with_bc-snow          0.3605109949745037  
                      aerosol-radiation_interactions  -0.0020748083021771557  
                      aerosol-cloud_interactions         0.40381198215439673  
                      aerosol-total                       0.4017371738522196  
...                                                                      ...  
ssp370-lowNTCF-gidden o3                                -0.09861654200626929  
                      HFCs                               0.08367522537429371  
                      bc_on_snow                       -0.027404367550451728  
                      total_anthropogenic                 2.6451235538746403  
                      total                                2.587185062896853  

[100 rows x 2 columns]

## Save output

In [20]:
tab.to_csv(PATH_DT_TAB_OUTPUT)

In [21]:
PATH_DT_TAB_OUTPUT

PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/results/tables_recommendation/table_sens_dT_cs_recommandetion.csv')